goal:
=====

basically a factorization machine with cross entropy loss where interaction effects come from deep nonlinear relu-activated embeddings and with an additional "metric" kernal matrix.

todo: dropout. currently no regularization on the interaction layers in the cost function. can handle with FTRL optimization

In [1]:
# import this stuff
import time
import sys
from pylab import *
from scipy import sparse
import numpy as np

import tensorflow as tf

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction import FeatureHasher
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import datasets
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix

In [2]:
def make_embeddings(x, rank, num_features, depth=1, seed=12345):
    """
      assumes that all hidden layers are width `rank`
    """
    assert depth > 0
    V = tf.Variable(tf.truncated_normal([rank, num_features], stddev=0.2, mean=0, seed=seed), name="v_1")
    b = tf.Variable(tf.truncated_normal([rank, 1], stddev=0.2, mean=0, seed=seed), name="b_1")
    Vx = tf.nn.relu(tf.matmul(V, x) + b)
    for i in range(depth - 1):
        V = tf.Variable(tf.truncated_normal([rank, rank], stddev=0.2, mean=0, seed=seed), name="v_%s" % i)
        b = tf.Variable(tf.truncated_normal([rank, 1], stddev=0.2, mean=0, seed=seed), name="b_%s" % i)
        Vx = tf.nn.relu(tf.matmul(V, Vx) + b)

    return Vx

def factorize(observed_features,
              labels,
              observed_features_validation,
              labels_validation,
              rank,
              max_iter=100,
              verbose=False,
              lambda_v=0,
              lambda_k=0,
              lambda_w=0,
              lambda_constants=0,
              epsilon=0.001,
              optimizer=tf.train.AdamOptimizer(),
              depth=3,
              seed=12345):

    # Extract info about shapes etc from the training data
    num_items = observed_features.shape[0]
    num_features = observed_features.shape[1]
    
    # matrix defining the inner product weights when doing interactions
    K = tf.Variable(tf.truncated_normal([rank, rank], stddev=0.2, mean=0, seed=seed), name="metric_matrix")
    
    # coefficients for linear function on inputs (wide part)
    w = tf.Variable(tf.truncated_normal([1, num_features], stddev=0.2, mean=0, seed=seed), name="hyperplane")

    # coefficients for linear functinos on inputs (deep part)
    lw = tf.Variable(tf.truncated_normal([1, rank], stddev=0.2, mean=0, seed=seed), name="latenthyperplane")

    # bias in linear function
    b = tf.Variable(tf.truncated_normal([1, 1], stddev=0.2, mean=0, seed=seed), name="b_one")
    
    x = tf.placeholder(tf.float32, [None, num_features])
    y = tf.placeholder(tf.float32)
    
    norm_x = tf.nn.l2_normalize(x, dim=0)
    
    Vx = make_embeddings(tf.transpose(norm_x), rank, num_features, depth=depth, seed=seed)
    right_kern = tf.matmul(K, Vx)
    
    full_kern = tf.matmul(tf.transpose(Vx), right_kern)
    linear = tf.matmul(w, tf.transpose(norm_x))
    latent_linear = tf.matmul(lw, Vx)

    pred = tf.reduce_sum(tf.sigmoid(linear + latent_linear + full_kern + b))
    
    # todo: dropout. currently no regularization on the interaction layers in the cost functino
    # can handle with FTRL optimization
    cost = tf.reduce_mean(-y*tf.log(pred + 0.0000000001) - (1-y)*tf.log((1-pred + 0.0000000001)) + 
            lambda_k*tf.nn.l2_loss(K) +
            lambda_w*tf.nn.l2_loss(w) +
            lambda_constants*tf.nn.l2_loss(b))
    optimize = optimizer.minimize(cost)
    norm = tf.reduce_mean(tf.nn.l2_loss(w))
    
    init = tf.initialize_all_variables()
    
    with tf.Session() as sess:
        sess.run(init)
        last_cost = 1000000
        for iter in range(0, max_iter):
            avg_cost = 0
            
            for i in range(num_items):
                _, c, n = sess.run([optimize, cost, norm],
                              feed_dict={x:observed_features[i].reshape(1, num_features), y:labels[i]})
                avg_cost += c / num_items
            if verbose:
                print("epoch: %s, cost: %s" % (iter+1, avg_cost))

            # check for convergence
            if abs(avg_cost-last_cost)/avg_cost < epsilon:
                break
                
            last_cost = avg_cost
            
        if verbose:
            print("optimization finished")
        predictions = []
        total_costs = 0
        for i in range(observed_features_validation.shape[0]):
            p, c = sess.run([pred, cost], feed_dict={x:observed_features_validation[i].reshape(1, num_features), y:labels_validation[i]})
            predictions.append(p)
            total_costs += c
        return predictions, total_costs/observed_features_validation.shape[0], sess.run([norm])

In [3]:
# use this data for now

categories = ['alt.atheism', 'soc.religion.christian', 'comp.graphics', 'sci.med']

ng = datasets.fetch_20newsgroups (categories=categories, shuffle=True)
labels = [1 if y == 2 else 0 for y in ng.target.reshape(-1,1)]

tfidf = TfidfVectorizer(decode_error=False, min_df=5)

X_train, X_test, y_train, y_test = train_test_split(ng.data, labels, test_size=.3)
X_train = tfidf.fit_transform(X_train).todense()
X_test = tfidf.transform(X_test).todense()


In [4]:
r = 10
predictions, test_costs, norm = factorize(X_train, y_train, X_test, y_test, r, verbose=True, lambda_v=0.1, max_iter=300)
print("rank: %s, cost: %s, overall AUC: %s, norm: %s") % (r, test_costs, roc_auc_score(y_test, predictions, average="weighted"), norm)

epoch: 1, cost: 0.364745618576
epoch: 2, cost: 0.0412070428173
epoch: 3, cost: 0.00511646193083
epoch: 4, cost: 0.000573719028131
epoch: 5, cost: 0.000103600946791
epoch: 6, cost: 2.48861599586e-05
epoch: 7, cost: 7.81363535431e-06
epoch: 8, cost: 2.3109170019e-06
epoch: 9, cost: 8.36582808002e-07
epoch: 10, cost: 3.1777204314e-07
epoch: 11, cost: 1.21777209178e-07
epoch: 12, cost: 4.61286348155e-08
epoch: 13, cost: 1.75530042967e-08
epoch: 14, cost: 6.37947406139e-09
epoch: 15, cost: 2.49139153724e-09
epoch: 16, cost: 9.43708823985e-10
epoch: 17, cost: 5.66225269191e-10
epoch: 18, cost: 7.54967001588e-11
epoch: 19, cost: 0.000651490314227
epoch: 20, cost: 1.26457385142e-08
epoch: 21, cost: 1.09470308829e-09
epoch: 22, cost: 5.66225503189e-10
epoch: 23, cost: 3.01986872635e-10
epoch: 24, cost: 1.88741786397e-10
epoch: 25, cost: 1.13245059238e-10
epoch: 26, cost: 7.54967001588e-11
epoch: 27, cost: 3.77483500794e-11
epoch: 28, cost: 3.77483500794e-11
optimization finished
rank: 10, cost:

## 

In [5]:
# with some regularization via the optimizer
r = 10
predictions, test_costs, norm = factorize(X_train, y_train, X_test, y_test, r, verbose=True, max_iter=30, optimizer=tf.train.FtrlOptimizer(1.0, l2_regularization_strength=1.0))
print("rank: %s, cost: %s, overall AUC: %s, norm: %s") % (r, test_costs, roc_auc_score(y_test, predictions, average="weighted"), norm)

epoch: 1, cost: 0.290899644598
epoch: 2, cost: 0.0197090622734
epoch: 3, cost: 0.00262073021407
epoch: 4, cost: 0.00191992069104
epoch: 5, cost: 0.00158168090376
epoch: 6, cost: 0.00136246844894
epoch: 7, cost: 0.0012049099984
epoch: 8, cost: 0.00108467415076
epoch: 9, cost: 0.000989169211804
epoch: 10, cost: 0.000911074729425


KeyboardInterrupt: 